In [9]:
# import all required libraries for Project as and when needed during the projects progress

import os
import pandas as pd
import numpy as np
import requests
from pathlib import Path
import matplotlib.pyplot as plt
from math import floor
import datetime as dt
import warnings
warnings.simplefilter('ignore', FutureWarning)

import sklearn
from sklearn.linear_model import LinearRegression

plt.rcParams['figure.figsize'] = (20,10)
plt.style.use('fivethirtyeight')
plt.show()

In [10]:
conda update pandas

Retrieving notices: ...working... DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/notices.json HTTP/1.1" 404 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/notices.json HTTP/1.1" 404 None
done

PackageNotInstalledError: Package is not installed in prefix.
  prefix: /Users/chetankothari/anaconda3
  package name: pandas



Note: you may need to restart the kernel to use updated packages.


In [11]:
#Read online zip file
df_zip = pd.read_csv('https://people.arcada.fi/~parland/hjd5_8amp_Gt3/EURUSD1m.zip')
df =df_zip.copy()
df["Datetimestamp"] = pd.to_datetime(df.Date.astype(str)+' '+df.Timestamp.astype(str))
df.drop(columns=['Date','Timestamp'],inplace=True)
df.set_index('Datetimestamp',inplace=True)
df.loc[df.index.weekday==6] = df.shift(-24)
df.head()
df.tail()

,Open,High,Low,Close,Volume
Datetimestamp,,,,,
2019-12-31 11:55:00,1.12266,1.12267,1.12250,1.12251,196.390000
2019-12-31 11:56:00,1.12250,1.12261,1.12247,1.12251,167.640000
2019-12-31 11:57:00,1.12254,1.12273,1.12254,1.12271,164.570002
2019-12-31 11:58:00,1.12271,1.12283,1.12262,1.12275,244.780001
2019-12-31 11:59:00,1.12276,1.12281,1.12272,1.12277,219.340002


In [12]:
# Dictionary for aggregations
agg_dict = {'Open': 'first',
          'High': 'max',
          'Low': 'min',
          'Close': 'last',
          'Volume': 'mean'}

# Aggregate
df1 = df.resample('D').mean()
df1 = df1.dropna()
df1.head()

,Open,High,Low,Close,Volume
Datetimestamp,,,,,
2010-01-01,1.433194,1.433288,1.433089,1.433178,60.243939
2010-01-03,1.431885,1.432103,1.431675,1.431898,24.579167
2010-01-04,1.436431,1.436693,1.436159,1.436451,55.569028
2010-01-05,1.440870,1.441108,1.440588,1.440881,55.477153
2010-01-06,1.436875,1.437116,1.436592,1.436898,56.230417


In [13]:
#Function for Williams %R Calculation
def find_LWR(high, low, close, lookback):
    highh = high.rolling(lookback).max() 
    lowl = low.rolling(lookback).min()
    lw = -100 * ((highh - close) / (highh - lowl))
    return lw

df1['LW%R'] = find_LWR(df1['High'], df1['Low'], df1['Close'], 14)
df1.tail()

,Open,High,Low,Close,Volume,LW%R
Datetimestamp,,,,,,
2019-12-26,1.109451,1.109475,1.109429,1.109451,36.627798,-79.562161
2019-12-27,1.114447,1.114498,1.114401,1.114452,94.375684,-17.066848
2019-12-29,1.118012,1.118057,1.117960,1.118008,39.328564,-0.481556
2019-12-30,1.119915,1.119964,1.119867,1.119915,90.878301,-0.401054
2019-12-31,1.120863,1.120901,1.120828,1.120866,61.131195,-0.270766


In [14]:
#Standardization with 80% of data rows

stdz_80 = int(0.8*len(df1.index))
stdz_mean = df1.head(stdz_80).mean()
stdz_std = df1.head(stdz_80).std()

df1_stdz = (df1 - stdz_mean)/stdz_std

df1_stdz["label"]=df1_stdz["Close"].shift(-1)

df1_stdz = df1_stdz.dropna()
df1_stdz

,Open,High,Low,Close,Volume,LW%R,label
Datetimestamp,,,,,,,
2010-01-17,1.587647,1.588005,1.587138,1.587703,-0.925410,-0.669819,1.599919
2010-01-18,1.599762,1.600399,1.598787,1.599919,-0.739638,-0.485457,1.562768
2010-01-19,1.562740,1.563538,1.561460,1.562768,-0.746704,-1.216398,1.413281
2010-01-20,1.413310,1.414248,1.411817,1.413281,-0.738785,-1.235549,1.351191
2010-01-21,1.351128,1.352218,1.349573,1.351191,-0.746857,-1.238339,1.390263
...,...,...,...,...,...,...,...
2019-12-25,-1.223294,-1.223908,-1.222696,-1.223287,-1.155373,-1.027106,-1.215440
2019-12-26,-1.215438,-1.216041,-1.214834,-1.215440,-0.907243,-0.733911,-1.172544
2019-12-27,-1.172584,-1.172951,-1.172172,-1.172544,-0.398887,0.868865,-1.142040


In [15]:
#drop Open, High, Low, Volume
df1_linear = df1_stdz[['Close','LW%R','label']]
df1_linear.head()

,Close,LW%R,label
Datetimestamp,,,
2010-01-17,1.587703,-0.669819,1.599919
2010-01-18,1.599919,-0.485457,1.562768
2010-01-19,1.562768,-1.216398,1.413281
2010-01-20,1.413281,-1.235549,1.351191
2010-01-21,1.351191,-1.238339,1.390263


In [16]:
# to update stdz_80 to 80% of df1_linear (NAN rows deleted)
stdz_80 = int(0.8*len(df1_linear.index))

# Now data is standardised, lets split X_train, y_train, X_test, y_test as below-
X_train = df1_linear.head(stdz_80)
X_train.drop('label', inplace=True, axis=1)

X_test = df1_linear.tail(len(df1_linear.index)-stdz_80)
X_test.drop('label', inplace=True, axis=1)

y_train = df1_linear['label'].head(stdz_80)
y_test = df1_linear['label'].tail(len(df1_linear.index)-stdz_80)
 
#create numpy arrays from the train, test dataframes
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()



[[ 1.58770258 -0.66981906]
 [ 1.59991934 -0.48545671]
 [ 1.56276823 -1.21639774]
 ...
 [-0.4581447   1.30051735]
 [-0.42972007  1.30062041]
 [-0.4023421   1.30114504]]
[ 1.59991934  1.56276823  1.41328081 ... -0.42972007 -0.4023421
 -0.41178316]


/var/folders/61/39zk8n_x5l32mb7rjkls_4n00000gn/T/ipykernel_3261/4203907780.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop('label', inplace=True, axis=1)
/var/folders/61/39zk8n_x5l32mb7rjkls_4n00000gn/T/ipykernel_3261/4203907780.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop('label', inplace=True, axis=1)


In [18]:
print(X_train)
print(y_train)

[[ 1.58770258 -0.66981906]
 [ 1.59991934 -0.48545671]
 [ 1.56276823 -1.21639774]
 ...
 [-0.4581447   1.30051735]
 [-0.42972007  1.30062041]
 [-0.4023421   1.30114504]]
[ 1.59991934  1.56276823  1.41328081 ... -0.42972007 -0.4023421
 -0.41178316]


In [ ]:
#create linear regression object
regr = LinearRegression()

In [17]:

#Train the model using the training sample
regr.fit(X_train, y_train)
display(regr)

#Calculate R-squared for both test / train samples
print('Training set score: ' + str(round(regr.score(X_train,y_train),4)))
print('Test set score: ' + str(round(regr.score(X_test,y_test),4)))

#Make predictions using the test sample
y_pred_scaled = regr.predict(X_test)
print(X_test.shape)
print(y_pred_scaled.shape)

LinearRegression()

Training set score: 0.9979
Test set score: 0.9943
(623, 2)
(623,)


In [13]:
#Inverse the scale back to the exchange rates
y_pred_lin = y_pred_scaled * stdz_std['Close'] + stdz_mean['Close']

In [14]:
#drop William's R and calculate linear regression again
X_train_C = X_train[:,0].reshape(-1,1)
X_test_C = X_test[:,0].reshape(-1,1)

#create linear regression object
regr_C = LinearRegression()

#Train the model using the training sample
regr_C.fit(X_train_C, y_train)
display(regr_C)
 
#Calculate R-squared for both test / train samples
print('Training set score: ' + str(round(regr_C.score(X_train_C,y_train),4)))
print('Test set score: ' + str(round(regr_C.score(X_test_C,y_test),4)))

#Make predictions using the test sample
y_pred_scaled_C = regr_C.predict(X_test_C)
print(X_test_C.shape)
print(y_pred_scaled_C.shape)

LinearRegression()

Training set score: 0.9965
Test set score: 0.9899
(623, 1)
(623,)
